In [18]:
import pdfplumber
import re
import pandas as pd

In [21]:
proveniences = pd.read_csv("../proveniences.csv")

In [30]:
prov_names = proveniences["provenience"].str.replace(r' \(mod\..*', '', regex = True).unique()

In [56]:
pdf = pdfplumber.open('ATU 3.pdf')

In [57]:
full_txt = ""
for page in pdf.pages:
    full_txt += page.extract_text(x_tolerance = 1) + " "

In [58]:
prov_occurence = []
for e in prov_names:
    pattern = r"\b" + e + r"\b"
    prov_occurence.append((e, len(re.findall(pattern, full_txt))))

In [59]:
prov_occurence.sort(key = lambda x: x[1], reverse = True)

In [60]:
prov_occurence

[('Uruk', 30),
 ('Ur', 4),
 ('Ebla', 2),
 ('uncertain', 1),
 ('Sippar-Amnanum', 0),
 ('Antipatris', 0),
 ('Qasr-i Abu Nasr', 0),
 ('Susa', 0),
 ('Dur-Kurigalzu', 0),
 ('Samal', 0),
 ('Lagaba', 0),
 ('Ba-milkišu', 0),
 ('Nippur', 0),
 ('Kutha', 0),
 ('Mardaman', 0),
 ('Sippar-Yahrurum', 0),
 ('Tuttul', 0),
 ('Surru', 0),
 ('Bit-zerija', 0),
 ('Ḫadatu', 0),
 ('Kapru', 0),
 ('Garšana', 0),
 ('Lagash', 0),
 ('Nigin', 0),
 ('GARIN-naḫallum', 0),
 ('Dusabar', 0),
 ('Gubla', 0),
 ('Irisagrig', 0),
 ('Tigunānum', 0),
 ('Shuruppak', 0),
 ('Upi', 0),
 ('Agamatanu', 0),
 ('Aqa', 0),
 ('Eridu', 0),
 ('Qatuna', 0),
 ('Naṣir', 0),
 ('Kesh', 0),
 ('Ṣurmeš', 0),
 ('Epurasie', 0),
 ('Kalhu', 0),
 ('Dūr-Abī-ēšuḫ', 0),
 ('Borsippa', 0),
 ('Tarbisu', 0),
 ('Kish', 0),
 ('Nineveh', 0),
 ('Šaḫrinu', 0),
 ('Girsu', 0),
 ('Unqi', 0),
 ('Ešnunna', 0),
 ('Pārśa', 0),
 ('Shaduppum', 0),
 ('Zabalam', 0),
 ('Kabnak', 0),
 ('Gišši', 0),
 ('Abdju', 0),
 ('Dur-Untaš', 0),
 ('Shaddikanni', 0),
 ('Seleucia', 0),
 ('Bit

In [41]:
article = nlp(full_txt)
len(article.ents)

542

In [27]:
items = list(filter(lambda x: x.label_ != 'CARDINAL' and x.label_ != 'ORDINAL', article.ents))

In [28]:
Counter(items)

Counter({Orientalistik
         Die: 1,
         Texte aus Uruk: 1,
         Archaische Texte aus Uruk, Band 3: 1,
         Ausgrabungen: 1,
         Deutschen Forschungsgemeinschaft: 1,
         Uruk-Warka: 1,
         Robert K.
         Englund: 1,
         Hans J. Nissen: 1,
         Peter Damerow
         Review: 1,
         Piotr Steinkeller: 1,
         1995/1996: 1,
         the Terms & Conditions of Use: 1,
         JSTOR: 1,
         JSTOR: 1,
         JSTOR: 1,
         4 Dec 2013: 1,
         10:57:06: 1,
         JSTOR Terms: 1,
         Robert K. Englund: 1,
         Hans J. Nissen: 1,
         Peter  Damerow: 1,
         Texte: 1,
         aus  Uruk: 1,
         contents),f: 1,
         Mann: 1,
         Verlag: 1,
         1993: 1,
         Archaische Texte: 1,
         aus  Uruk: 1,
         distributiono: 1,
         Deutschen Forschungsgemeinschafitn: 1,
         ransliteratioonf: 1,
         ED: 1,
         greateri nterestt: 1,
         y P. Damerow: 1,
         num

In [30]:
sentences = [x for x in article.sents]
displacy.render(nlp(str(sentences[0: 100])), jupyter=True, style='ent')